<h1>Spark SQL, DataFrames and Datasets Guide</h1>
<ul>
  <li>Overview<ul>
      <li>SQL</li>
      <li>Datasets and DataFrames</li>
    </ul>
  </li>
  <li>Getting Started<ul>
      <li>Starting Point: SparkSession</li>
      <li>Creating DataFrames</li>
      <li>Untyped Dataset Operations(aka DataFrame Operations)</li>
      <li>Running SQL Queries Programmatically</li>
      <li>Creating Datasets</li>
      <li>Interoperating with RDDs<ul>
          <li>Inferring the Schema Using Reflection</li>
          <li>Programmatically Specifying the Schema</li>
        </ul>
      </li>
    </ul>
  </li>
  <li>Data Sources<ul>
      <li>Generic Load/Save Functions<ul>
          <li>Manually Specifying Options</li>
          <li>Run SQL on files directly</li>
          <li>Save Modes</li>
          <li>Saving to Persistent Tables</li>
        </ul>
      </li>
      <li>Parquet Files<ul>
          <li>Loading Data Programmatically</li>
          <li>Partition Discovery</li>
          <li>Schema Merging</li>
          <li>Hive metastore Parquet table conversion<ul>
              <li>Hive/Parquet Schema Reconciliation</li>
              <li>Metadata Refreshing</li>
            </ul>
          </li>
          <li>Configuration</li>
        </ul>
      </li>
      <li>JSON Datasets</li>
      <li>Hive Tables<ul>
          <li>Interacting with Different Versions of Hive Metastore</li>
        </ul>
      </li>
      <li>JDBC To Other Databases</li>
      <li>Troubleshooting</li>
    </ul>
  </li>
  <li>Performance Tuning<ul>
      <li>Caching Data In Memory</li>
      <li>Other Configuration Options</li>
    </ul>
  </li>
  <li>Distributed SQL Engine<ul>
      <li>Running the Thrift JDBC/ODBC server</li>
      <li>Running the Spark SQL CLI</li>
    </ul>
  </li>
  <li>Migration Guide<ul>
      <li>Upgrading From Spark SQL 1.6 to 2.0</li>
      <li>Upgrading From Spark SQL 1.5 to 1.6</li>
      <li>Upgrading From Spark SQL 1.4 to 1.5</li>
      <li>Upgrading from Spark SQL 1.3 to 1.4 <ul>
          <li>DataFrame data reader/writer interface</li>
          <li>DataFrame.groupBy retains grouping columns</li>
          <li>Behavior change on DataFrame.withColumn</li>
        </ul>
      </li>
      <li>Upgrading from Spark SQL 1.0-1.2 to 1.3<ul>
          <li>Rename of SchemaRDD to DataFrame</li>
          <li>Unification of the Java and Scala APIs</li>
          <li>Isolation of Implicit Conversions and Removal of dsl Package (Scala-only)</li>
          <li>Removal of the type aliases in org.apache.spark.sql for DataType (Scala-only)</li>
          <li>UDF Registration Moved to <code>sqlContext.udf</code> (Java &amp; Scala)</li>
          <li>Python DataTypes No Longer Singletons</li>
        </ul>
      </li>
      <li>Compatibility with Apache Hive<ul>
          <li>Deploying in Existing Hive Warehouses</li>
          <li>Supported Hive Features</li>
          <li>Unsupported Hive Functionality</li>
        </ul>
      </li>
    </ul>
  </li>
  <li>Reference    <ul>
      <li>Data Types</li>
      <li>NaN Semantics</li>
    </ul>
  </li>
</ul>

<h1>Overview</h1>
<p>Spark SQL is a Spark module for structured data processing. Unlike the basic Spark RDD API, the interfaces provided
by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally,Spark SQL uses this extra information to perform extra optimizations. There are several ways to
interact with Spark SQL including SQL and the Dataset API. When computing a result the same execution engine is used, independent of which API/language you are using to express the computation. This unification(统一，联合; 一致;
) means that developers can easily switch back and forth between different APIs based on which provides the most natural way to express a given transformation.</p>
<p>All of the examples on this page use sample data included in the Spark distribution and can be run in the <code>spark-shell</code>, <code>pyspark</code> shell, or <code>sparkR</code> shell.</p>

<h2>SQL</h2>
<p>One use of Spark SQL is to execute SQL queries.Spark SQL can also be used to read data from an existing Hive installation. For more on how to configure this feature, please refer to the <strong>Hive Tables</strong> section. When running SQL from within another programming language the results will be returned as a <strong>Dataset/DataFrame</strong>.You can also interact with the SQL interface using the <strong>command-line</strong> or over <strong>JDBC/ODBC</strong>.</p>

<h2>Datasets and DataFrames</h2>

<p>A Dataset is a distributed collection of data.Dataset is a new interface added in Spark 1.6 that provides the benefits of RDDs (strong typing, ability to use powerful lambda functions) with the benefits of Spark SQL&#8217;s optimized execution engine. A Dataset can be <strong>constructed</strong> from JVM objects and then
manipulated using functional transformations (<code>map</code>, <code>flatMap</code>, <code>filter</code>, etc.).
The Dataset API is available in <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset">Scala</a> and
<a href="http://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/Dataset.html">Java</a>. Python does not have the support for the Dataset API. But due to Python&#8217;s dynamic nature,many of the benefits of the Dataset API are already available (i.e. you can access the field of a row by name naturally<code>row.columnName</code>). The case for R is similar.</p>

<p>A DataFrame is a <em>Dataset</em> organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood(在幕后). DataFrames can be constructed from a wide array of <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html#data-sources">sources</a> such as: <u>structured data files, tables in Hive, external databases, or existing RDDs</u>.The DataFrame API is available in Scala,Java, <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">Python</a>, and <a href="http://spark.apache.org/docs/latest/api/R/index.html">R</a>.In Scala and Java, a DataFrame is represented by a Dataset of <code>Row</code>s.In <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset">the Scala API</a>, <code>DataFrame</code> is simply a type alias of <code>Dataset[Row]</code>.While, in <a href="http://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/Dataset.html">Java API</a>, users need to use <code>Dataset&lt;Row&gt;</code> to represent a <code>DataFrame</code>.</p>

<p>Throughout this document, we will often refer to Scala/Java Datasets of <code>Row</code>s as DataFrames.</p>

<h1>Getting Started</h1>
<h2>Starting Point: SparkSession</h2>
<p>The entry point into(入口点) all functionality in Spark is the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession"><code>SparkSession</code></a> class. To create a basic <code>SparkSession</code>, just use <code>SparkSession.builder</code>:</p>

In [1]:
from pyspark.sql import SparkSession
import shutil
spark = SparkSession\
.builder\
.appName("PythonSQL")\
.config("spark.some.config.option", 'some-value')\
.getOrCreate()

<p><code>SparkSession</code> in Spark 2.0 provides builtin support for Hive features including the ability to write queries using HiveQL, access to Hive UDFs(User defined Function,用户自定义函数), and the ability to read data from Hive tables.To use these features, you do not need to have an existing Hive setup.</p>

<h2 id="creating-dataframes">Creating DataFrames</h2>
<p>With a <code>SparkSession</code>, applications can create DataFrames from an <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html#interoperating-with-rdds">existing <code>RDD</code></a>,from a Hive table, or from <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html#data-sources">Spark data sources</a>.</p>

<p>As an example, the following creates a DataFrame based on the content of a JSON file:</p>

In [2]:
# spark is an existing SparkSession
df = spark.read.json("resources/people.json")
# Displays the content of the DataFrame to stdout
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<h2>Untyped Dataset Operations (aka DataFrame Operations)</h2>
<p>DataFrames provide a domain-specific language(DSL:领域特定语言) for structured data manipulation in <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset">Scala</a>, <a href="https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/Dataset.html">Java</a>, <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">Python</a> and <a href="https://spark.apache.org/docs/latest/api/R/DataFrame.htmll">R</a>.</p>
<p>As mentioned above, in Spark 2.0, DataFrames are just Dataset of <code>Row</code>s in Scala and Java API. These operations are also referred as “untyped transformations” in contrast to “typed transformations” come with strongly typed(强类型) Scala/Java Datasets.</p>
<p>Here we include some basic examples of structured data processing using Datasets:</p>
<p>In Python it’s possible to access a DataFrame’s columns either by attribute (<code>df.age</code>) or by indexing (<code>df['age']</code>). While the former(前者) is convenient for interactive data exploration, users are highly encouraged to use the latter form, which is future proof and won’t break with column names that are also attributes on the DataFrame class.</p>

In [3]:
# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [4]:
# Select only the "name" column
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [5]:
# Select everybody, but increment the age by 1
df.select(df["name"], df["age"] + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [6]:
# Select people older than 21
df.filter(df['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [7]:
# Count people by age
df.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



<p>For a complete list of the types of operations that can be performed on a DataFrame refer to the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">API Documentation</a>.</p>
<p>In addition to simple column references and expressions, DataFrames also have a rich library of functions including string manipulation, date arithmetic, common math operations and more. The complete list is available in the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions">DataFrame Function Reference</a>.</p>
<h2>Running SQL Queries Programmatically(编码的方式运行SQL查询)</h2>
<ul>
<li><b>Scala</b></li><li><b>Java</b></li>
<li><b>Python</b></li><li><b>R</b></li>
</ul>
<p>The <code>sql</code> function on a <code>SparkSession</code> enables applications to run SQL queries programmatically and returns the result as a <code>DataFrame</code>.</p>

<dl>
<dt>
<a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createorreplacetempview#pyspark.sql.DataFrame.createOrReplaceTempView" ><tt>createOrReplaceTempView</tt><big>(</big><em>name</em><big>)</big></a></dt>
<dd><p>Creates or replaces a temporary view with this DataFrame.</p>
<p>The lifetime of this temporary table is tied to the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createorreplacetempview#pyspark.sql.SparkSession" target="_blank"><tt>SparkSession</tt></a>
that was used to create this <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createorreplacetempview#pyspark.sql.DataFrame"><tt>DataFrame</tt></a>.</p>
</dd></dl>

In [8]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")
sqlDF = spark.sql("select * from people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<h2>Creating Datasets(仅支持Scala,Java)</h2>
<p>Datasets are similar to RDDs, however, instead of using Java serialization or Kryo they use
a specialized <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Encoder">Encoder</a> to serialize the objects
for processing or transmitting over the network. While both encoders and standard serialization are
responsible for turning an object into bytes, encoders are code generated dynamically and use a format
that allows Spark to perform many operations like filtering, sorting and hashing without deserializing
the bytes back into an object.</p>

<h2>Interoperating with(互操作) RDDs</h2>

<p>Spark SQL supports two different methods for converting existing RDDs into Datasets. The first method uses reflection(反射) to infer(推断; 猜想) the schema of an RDD that contains specific types of objects. This reflection based approach leads to more concise code(简洁的代码) and works well when you already know the schema while writing your Spark application.</p>
<p>The second method for creating Datasets is through a programmatic interface that allows you to construct a schema and then apply it to an existing RDD. While this method is more verbose(冗长的，啰唆的，累赘的;), it allows you to construct Datasets when the columns and their types are not known until runtime.</p>

<h3>Inferring the Schema Using Reflection</h3>
<p>Spark SQL can convert an RDD of <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=row#pyspark.sql.Row" target="_blank">Row</a> objects to a DataFrame, inferring the datatypes. Rows are constructed by passing a list of key/value pairs as kwargs to the Row class. The keys of this list define the column names of the table, and the types are inferred by sampling the whole datase, similar to the inference that is performed on JSON files.</p>

In [9]:
from pyspark.sql import Row
# Load a text file and convert each line to a Row.
lines = sc.textFile("resources/people.txt")
lines.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [10]:
parts = lines.map(lambda line: line.split(","))
parts.collect()

[['Michael', ' 29'], ['Andy', ' 30'], ['Justin', ' 19']]

In [11]:
people = parts.map(lambda people: Row(name=people[0], age=int(people[1])))
people.collect()

[Row(age=29, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [12]:
# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")
# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")
# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
    print(name)

Name: Justin


<h3>Programmatically Specifying the Schema</h3>
<p>When case classes cannot be defined ahead of time (for example,the structure of records is encoded in a string, or a text dataset will be parsed and fields will be projected differently for different users),a <code>DataFrame</code> can be created programmatically with three steps.</p>
<ol>
  <li>Create an RDD of <code>Row</code>s from the original RDD;</li>
  <li>Create the schema represented by a <a  href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=structtype#pyspark.sql.types.StructType" target="_blank"><code>StructType</code></a> matching the structure of
<code>Row</code>s in the RDD created in Step 1.</li>
  <li>Apply the schema to the RDD of <code>Row</code>s via <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createdataframe#pyspark.sql.SparkSession.createDataFrame" target="_blank"><code>createDataFrame</code></a> method provided by <code>SparkSession</code>.</li>
</ol>
<p>For example:</p>

In [13]:
# Import data types
from pyspark.sql.types import *
# Load a text file and convert each line to a Row.
lines = sc.textFile("resources/people.txt")
parts = lines.map(lambda line: line.split(","))
# Each line is converted to a tuple.
people = parts.map(lambda people: (people[0], people[1].strip()))
# The schema is encoded in a string.
schemaString = "name age"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
print("fields:", fields, '\n')
schema = StructType([StructField("name", StringType(), True), StructField("age", StringType(), True)])
print("schema:", schema, '\n')
# Apply the schema to the RDD.
schemaPeople = spark.createDataFrame(people, schema)
# Creates a temporary view using the DataFrame
schemaPeople.createOrReplaceTempView("people")
# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT name FROM people")
results.show()

fields: [StructField(name,StringType,true), StructField(age,StringType,true)] 

schema: StructType(List(StructField(name,StringType,true),StructField(age,StringType,true))) 

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



<h1>Data Sources</h1>

<p>Spark SQL supports operating on a variety of data sources through the DataFrame interface.A DataFrame can be operated on using relational transformations and can also be used to create a temporary view.<strong>Registering a DataFrame as a temporary view allows you to run SQL queries over its data</strong>. This section describes the general methods for loading and saving data using the Spark Data Sources and then goes into specific options that are available for the built-in data sources.</p>

<h2>Generic Load/Save Functions</h2>

<p>In the simplest form, the default data source(<code>parquet</code> <u>unless otherwise configured</u>(除非另外配置) by <code>spark.sql.sources.default</code>) will be used for all operations.</p>

In [14]:
try:
    shutil.rmtree("resources/namesAndFavColors.parquet")
except FileNotFoundError:
    pass

df = spark.read.load("resources/users.parquet")
df.select("name","favorite_color").write.save("resources/namesAndFavColors.parquet")

In [15]:
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



<h3>Manually Specifying Options</h3>

<p>You can also manually specify the data source that will be used along with any extra options that you would like to pass to the data source. Data sources are specified by their fully qualified name (i.e., <code>org.apache.spark.sql.parquet</code>), but for built-in sources you can also use their short names (<code>json</code>, <code>parquet</code>, <code>jdbc</code>). <strong>DataFrames loaded from any data source type can be converted into other types using this syntax</strong>.</p>

In [16]:
try:
    shutil.rmtree("resources/namesAndAges.parquet")
except FileNotFoundError:
    pass

df = spark.read.load("resources/people.json", format="json")
df.select("name", "age").write.save("resources/namesAndAges.parquet", format="parquet")

<h3>Run SQL on files directly</h3>

<p>Instead of using read API to load a file into DataFrame and query it, you can also query that file directly with SQL.</p>

In [17]:
df = spark.sql("SELECT * FROM parquet.`resources/users.parquet`")
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



<h3>Save Modes</h3>

<p>Save operations can optionally take a <code>SaveMode</code>, that specifies how to handle existing data if
present. <font color="red"><strong>It is important to realize that these save modes do not utilize(利用;运用;使用;应用) any locking and are not atomic. Additionally, when performing an <code>Overwrite</code>, the data will be deleted before writing out the new data</strong>.</font></p>

<table>
<tr><th>Scala/Java</th><th>Any Language</th><th>Meaning</th></tr>
<tr>
  <td><code>SaveMode.ErrorIfExists</code> (default)</td>
  <td><code>"error"</code> (default)</td>
  <td>
    When saving a DataFrame to a data source, if data already exists,an exception is expected to be thrown.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Append</code></td>
  <td><code>"append"</code></td>
  <td>
    When saving a DataFrame to a data source, if data/table already exists,contents of the DataFrame are expected to be appended to existing data.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Overwrite</code></td>
  <td><code>"overwrite"</code></td>
  <td>
    Overwrite mode means that when saving a DataFrame to a data source,if data/table already exists, existing data is expected to be overwritten by the contents of the DataFrame.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Ignore</code></td>
  <td><code>"ignore"</code></td>
  <td>
    Ignore mode means that when saving a DataFrame to a data source, if data already exists,the save operation is expected to not save the contents of the DataFrame and to not change the existing data. This is similar to a <code>CREATE TABLE IF NOT EXISTS</code> in SQL.
  </td>
</tr>
</table>

<h3>Saving to Persistent Tables</h3>

<p><code>DataFrames</code> can also be saved as persistent tables into Hive metastore using the <strong><code>saveAsTable</code></strong>
command. Notice existing Hive deployment is not necessary to use this feature. Spark will create a default local Hive metastore (using Derby) for you. Unlike the <strong><code>createOrReplaceTempView</code></strong> command,<strong><code>saveAsTable</code></strong> will materialize(实现) the contents of the DataFrame and create a pointer to the data in the Hive metastore. Persistent tables will still exist even after your Spark program has restarted, as long as you maintain your connection to the same metastore. A DataFrame for a persistent table can be created by calling the <strong><code>table</code></strong> method on a <strong><code>SparkSession</code></strong> with the name of the table.</p>

<p>By default <strong><code>saveAsTable</code></strong> will create a &#8220;managed table&#8221;, meaning that the location of the data will be controlled by the metastore. Managed tables will also have their data deleted automatically when a table is dropped.</p>
<p>在Hive上有两种类型的表，一种是Managed Table，另一种是External Table。它俩的主要区别在于：当我们drop表时，Managed Table会同时删去data和meta data，而External Table只会删meta data。</p>
<h2>Parquet Files</h2>

<p><a href="http://parquet.io">Parquet</a> is a columnar format that is supported by many other data processing systems.Spark SQL provides support for both reading and writing Parquet files that automatically preserves(保留；保护；保存；维护) the schema of the original data. When writing Parquet files, all columns are automatically converted to be nullable(允许空值) for compatibility reasons(出于兼容方面的考虑).</p>

<h3>Loading Data Programmatically</h3>

<p>Using the data from the above example:</p>

In [18]:
try:
    shutil.rmtree("resources/people.parquet")
except FileNotFoundError:
    pass

peopleDF = spark.read.json("resources/people.json")

# DataFrames can be saved as Parquet files, maintaining the schema information.
peopleDF.write.parquet("resources/people.parquet")

# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("resources/people.parquet")
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



<p>Parquet是面向分析型业务的列式存储格式,对于优化『关系型数据库上的分析任务』，列式存储(Columnar Storage)是个比较流行的技术。我们的目标是，对于一个查询，尽量只读取对这个查询有用的数据，以此来让磁盘IO最小，做到了『指哪打哪』，也就是遍历（scan）一个数据集的时候，如果只读取部分列，那么读取时间也相应会缩短，时间缩短的比例就是那几列的数据量占全部列数据量的比例.原理很简单，就是不采用传统的按行存储，而是连续存储一列的数据. 如果数据是扁平的（比如二维表形式），那列改成按列存储毫无难度，处理嵌套的数据结构才是真正的挑战.开源项目 Parquet 是 Hadoop 上的一种支持列式存储文件格式，起初只是 Twitter 和 Coudera 在合作开发，发展到现在已经有包括 Criteo公司 在内的许多其他贡献者了.</p>
<p>何谓列式存储？看下面的例子，这就是三个列 A B C.</p>
<img src="SparkMLlib/imgs/Parquet_01.png">
<p>如果把它换成行式存储的，那么数据就是一行挨着一行存储的:</p>
<img src="SparkMLlib/imgs/Parquet_02.png">
<p>按列存，有几个好处：</p>
<ul>
  <li>可以跳过不符合条件的数据，只读取需要的数据，降低IO数据量.</li>
  <li>只读取需要的列，支持向量运算，能够获取更好的扫描性能.</li>
  <li>按列存，能够更好地压缩数据，因为一列的数据一般都是同质的（homogenous）. 对于hadoop集群来说，空间节省非常可观.</li>
  <li>I/O 会大大减少，因为扫描（遍历/scan）的时候，可以只读其中部分列. 而且由于数据压缩的更好的缘故，IO所需带宽也会减小.</li>
  <li>由于每列存的数据类型是相同的，we can use encodings better suited to the modern processors’ pipeline by making instruction branching more predictable.</li>
</ul>
<p>更多详情请点击<a href="http://lastorder.me/tag/parquet.html" target="_blank">这里</a>以及<a href="http://www.infoq.com/cn/articles/in-depth-analysis-of-parquet-column-storage-format/" target="_blank">这里</a>。</p>

<h3>Partition Discovery(解析分区信息)</h3>
<p>Table partitioning(表分区) is a common optimization approach(优化方法) used in systems like Hive. In a partitioned table(分区表), data are usually stored in different directories, with partitioning column values encoded(编码) in the path of each partition directory. The Parquet data source is now able to discover and infer partitioning information automatically. For example, we can store all our previously used population data into a partitioned table using the following directory structure, with two extra columns, <code>gender</code>(性别) and <code>country</code>(国家) as partitioning columns:</p>
<p>类似hive的分区表，在分区表中数据会分开存储在不同的文件夹，区分的标准是分区字段，现在parquet的数据源可以自动的发现分区信息.</p>
<pre><strong>path
└── to
    └── table
        ├── gender=male
        │   ├── ...
        │   │
        │   ├── country=US
        │   │   └── data.parquet
        │   ├── country=CN
        │   │   └── data.parquet
        │   └── ...
        └── gender=female
            ├── ...
            │
            ├── country=US
            │   └── data.parquet
            ├── country=CN
            │   └── data.parquet
            └── ...</strong></pre>
<p>By passing <strong>path/to/table</strong> to either <strong>SparkSession.read.parquet</strong> or <strong>SparkSession.read.load</strong>, Spark SQL will automatically extract the partitioning information from the paths.Now the schema of the returned DataFrame becomes:</p>
<p>通过传递<strong>path/to/table</strong>给<strong>SparkSession.read.parquet</strong>或<strong>SparkSession.read.load</strong>，Spark SQL将自动解析分区信息。返回的DataFrame的Schema如下：</p>
<pre><strong>root
|-- name: string (nullable = true)
|-- age: long (nullable = true)
|-- gender: string (nullable = true)
|-- country: string (nullable = true)</strong></pre>
<p>Notice that the data types of the partitioning columns are automatically inferred. Currently, numeric data types and string type are supported. Sometimes users may not want to automatically infer the data types of the partitioning columns. For these use cases, the automatic type inference can be configured by <strong>spark.sql.sources.partitionColumnTypeInference.enabled</strong>, which is default to <strong>true</strong>. When type inference is disabled, string type will be used for the partitioning columns.</p>
<p>需要注意的是，数据的分区列的数据类型是自动解析的。当前，支持数值类型和字符串类型。自动解析分区类型的参数为：<strong>spark.sql.sources.partitionColumnTypeInference.enabled</strong>，默认值为true。如果想关闭该功能，直接将该参数设置为disabled。此时，分区列数据格式将被默认设置为string类型，不再进行类型解析。</p>
<p>Starting from Spark 1.6.0, partition discovery only finds partitions under the given paths by default. For the above example, if users pass <strong>path/to/table/gender=male</strong> to either <strong>SparkSession.read.parquet</strong> or <strong>SparkSession.read.load</strong>, <strong>gender</strong> will not be considered as a partitioning column. If users need to specify the base path that partition discovery should start with, they can set <strong>basePath</strong> in the data source options. For example, when <strong>path/to/table/gender=male</strong> is the path of the data and users set <strong>basePath</strong> to <strong>path/to/table/</strong>, <strong>gender</strong> will be a partitioning column.</p>
<p>从1.6版本开始，如果用了分区路径，那么会丢失用于分区路径的那个字段，只能通过basedir拿到，而把分区属性变成分区字段.</p>

In [19]:
spark.read.parquet("resources/people.parquet").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<dl>
<a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=partitionby#pyspark.sql.DataFrameWriter.partitionBy" target="_blank">
<dt>
<tt>partitionBy</tt><big>(</big><em>*cols</em><big>)</big></dt></a>
<dd><p>Partitions the output by the given columns on the file system.</p>
<p>If specified, the output is laid out on the file system similar
to Hive’s partitioning scheme.</p>
<table rules="none">
<colgroup><col class="field-name">
<col class="field-body">
</colgroup><tbody valign="top">
<tr class="field-odd field"><th class="field-name">Parameters:</th><td class="field-body"><strong>cols</strong> – name of columns</td>
</tr>
</tbody>
</table>
<div class="highlight-python"><div class="highlight"><pre><span class="gp">&gt;&gt;&gt; </span><span class="n">df</span><span class="o">.</span><span class="n">write</span><span class="o">.</span><span class="n"><span class="highlighted">partitionBy</span></span><span class="p">(</span><span class="s">'year'</span><span class="p">,</span> <span class="s">'month'</span><span class="p">)</span><span class="o">.</span><span class="n">parquet</span><span class="p">(</span><span class="n">os</span><span class="o">.</span><span class="n">path</span><span class="o">.</span><span class="n">join</span><span class="p">(</span><span class="n">tempfile</span><span class="o">.</span><span class="n">mkdtemp</span><span class="p">(),</span> <span class="s">'data'</span><span class="p">))</span>
</pre></div>
</div>
</dd></dl>
<p>从这个函数我们可以看出按照我们给出的字段完成了一个partition的过程，接受一个多参数的字段列表.</p>
<font color="red">关于Partition Discovery更多详情请参考<a href="http://blog.csdn.net/cjuexuan/article/details/51165310" target="_blank">这篇博文</a>。</font>

In [20]:
try:
    shutil.rmtree("resources/peoplePartitionBy.parquet")
except FileNotFoundError:
    pass

spark.read.json("resources/people.json").write.partitionBy("name").parquet("resources/peoplePartitionBy.parquet")

<h3>查看输出路径下的目录：</h3>
<pre><code >_SUCCESS      name=Michael        name=Justin        name=Andy</code></pre>

<p>我们可以发现按照我们的分区规则，在输出路径上建立了三个文件夹，分别为name=Michael,name=Justin,name=Andy</p>
<h3>读取：</h3>

In [21]:
spark.read.parquet("resources/peoplePartitionBy.parquet/name=Andy").show()

+---+
|age|
+---+
| 30|
+---+



In [22]:
spark.read.parquet("resources/peoplePartitionBy.parquet/name=Michael").show()

+----+
| age|
+----+
|null|
+----+



In [23]:
spark.read.parquet("resources/peoplePartitionBy.parquet/name=Justin").show()

+---+
|age|
+---+
| 19|
+---+



In [24]:
spark.read.parquet("resources/peoplePartitionBy.parquet").show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



<dl>
<dt>
<tt>filter</tt><big>(</big><em>condition</em><big>)</big><a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.DataFrame.filter" target="_blank">¶</a></dt>
<dd><p>Filters rows using the given condition.</p>
<p><a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.DataFrame.where"><tt>where()</tt></a> is an alias for <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.DataFrame.filter"><tt>filter()</tt></a>.</p>
<table frame="void" rules="none">
<colgroup><col>
<col>
</colgroup><tbody valign="top">
<tr><th>Parameters:</th><td><strong>condition</strong> – a <a  href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.Column" ><tt>Column</tt></a> of <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.types.BooleanType" ><tt>types.BooleanType</tt></a>
or a string of SQL expression.</td>
</tr>
</tbody>
</table>
<pre>&gt;&gt;&gt; df.filter(.age&gt;3).collect()
[Row(age=5, name=u'Bob')]
&gt;&gt;&gt; df.where(df.age==2).collect()
[Row(age=2, name=u'Alice')]
</pre>
<pre>&gt;&gt;&gt; df.filter("age &gt; 3").collect()
[Row(age=5, name=u'Bob')]
&gt;&gt;&gt; df.where("age = 2").collect()
[Row(age=2, name=u'Alice')]
</pre>
</dd></dl>

In [25]:
df = spark.read.parquet("resources/peoplePartitionBy.parquet")
df.filter(df.name == "Andy").collect()

[Row(age=30, name='Andy')]

In [26]:
spark.read.parquet("resources/peoplePartitionBy.parquet").filter('name="Andy"').show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



<h3>Schema Merging</h3>
<p>Like ProtocolBuffer, Avro, and Thrift, Parquet also supports schema evolution(演变; 进化; 发展;). Users can start with a simple schema, and gradually add more columns to the schema as needed. In this way, users may end up with multiple Parquet files with different but mutually compatible(相互兼容) schemas. The Parquet data source is now able to automatically detect this case and merge schemas of all these files.</p>

<p>Since schema merging is a relatively `expensive operation`, and is not a necessity in most cases, we turned it off by default starting from 1.5.0. You may enable it by</p>
<ol>
  <li>setting data source <font color="red">option</font> <strong>mergeSchema</strong> to <code>true</code> when reading Parquet files (as shown in the examples below), or</li>
  <li>setting the global SQL option <strong>`spark.sql.parquet.mergeSchema`</strong> to <code>true</code>.</li>
</ol>

In [27]:
import os

if os.path.exists("data/test_table") == False:
    os.makedirs("data/test_table")

try:
    shutil.rmtree("data/test_table")
except FileNotFoundError:
    pass

# Create a simple DataFrame, stored into a partition directory
squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6)).map(lambda i: Row(single=i, double=i ** 2)))
squaresDF.write.parquet("data/test_table/key=1")
# Create another DataFrame in a new partition directory,
# adding a new column and dropping an existing column
cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11)).map(lambda i: Row(single=i, triple=i ** 3)))
cubesDF.write.parquet("data/test_table/key=2")
# Read the partitioned table
mergedDF = spark.read.option("mergeSchema", 'true').parquet("data/test_table")
mergedDF.printSchema()
# The final schema consists of all 3 columns in the Parquet files together
# with the partitioning column appeared in the partition directory paths.
# root
#  |-- double: long (nullable = true)
#  |-- single: long (nullable = true)
#  |-- triple: long (nullable = true)
#  |-- key: integer (nullable = true)

root
 |-- double: long (nullable = true)
 |-- single: long (nullable = true)
 |-- triple: long (nullable = true)
 |-- key: integer (nullable = true)



<h3>Hive metastore Parquet table conversion</h3>

<p>When reading from and writing to Hive metastore Parquet tables, Spark SQL will try to use its own Parquet support instead of Hive SerDe(Serialize/Deserilize的简称,目的是用于序列化和反序列化.) for better performance. This behavior is controlled by the <strong>spark.sql.hive.convertMetastoreParquet</strong> configuration, and is turned on by default.</p>

<h4>Hive/Parquet Schema Reconciliation(一致化)</h4>
<p>There are two key differences between Hive and Parquet from the perspective(角度) of table schema processing.</p>
<ol>
  <li>Hive is case insensitive, while Parquet is not(Hive区分大小写，Parquet不区分大小写)</li>
  <li>Hive considers all columns nullable(允许为空,可空类型), while nullability(空属性) in Parquet is significant</li>
</ol>
<p>Due to this reason, we must reconcile(兼顾:使和谐一致;调和;使配合) Hive metastore schema with Parquet schema when converting a Hive metastore Parquet table to a Spark SQL Parquet table. The reconciliation rules are:</p>
<ol>
  <li>
    <p>Fields that have the same name in both schema must have the same data type regardless of nullability(空属性). The reconciled field should have the data type of the Parquet side, so that nullability is respected.</p>
  </li>
  <li>
    <p>The reconciled schema contains exactly those fields defined in Hive metastore schema.</p>
    <ul>
      <li>Any fields that only appear in the Parquet schema are dropped in the reconciled schema.
      </li>
      <li>Any fields that only appear in the Hive metastore schema are added as nullable field in the reconciled schema.
      </li>
    </ul>
  </li>
</ol>

<h4>Metadata Refreshing</h4>
<p>Spark SQL caches Parquet metadata for better performance. When Hive metastore Parquet table conversion is enabled, metadata of those converted tables are also cached. If these tables are updated by Hive or other external tools, you need to refresh them manually to ensure consistent metadata.</p>

In [28]:
#spark.catalog.refreshTable("my_table")

<h3>Configuration</h3>

<p>Configuration of Parquet can be done using the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=setconf#pyspark.sql.SQLContext.setConf" target="_blank"><code>setConf</code></a> method on <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=setconf#pyspark.sql.SparkSession" target="_blank"><code>SparkSession</code></a> or by running <code>SET key=value</code> commands using SQL.</p>

<table>
<tr><th>Property Name</th><th>Default</th><th>Meaning</th></tr>
<tr>
  <td><code>spark.sql.parquet.binaryAsString</code></td>
  <td>false</td>
  <td>
    Some other Parquet-producing systems, in particular Impala, Hive, and older versions of Spark SQL, do not differentiate between binary data and strings when writing out the Parquet schema. This flag tells Spark SQL to interpret binary data as a string to provide compatibility with these systems.
  </td>
</tr>
<tr>
  <td><code>spark.sql.parquet.int96AsTimestamp</code></td>
  <td>true</td>
  <td>
    Some Parquet-producing systems, in particular Impala and Hive, store Timestamp into INT96. This flag tells Spark SQL to interpret INT96 data as a timestamp to provide compatibility with these systems.
  </td>
</tr>
<tr>
  <td><code>spark.sql.parquet.cacheMetadata</code></td>
  <td>true</td>
  <td>
    Turns on caching of Parquet schema metadata. Can speed up querying of static data.
  </td>
</tr>
<tr>
  <td><code>spark.sql.parquet.compression.codec</code></td>
  <td>snappy</td>
  <td>
    Sets the compression codec use when writing Parquet files. Acceptable values include:uncompressed, snappy, gzip, lzo.
  </td>
</tr>
<tr>
  <td><code>spark.sql.parquet.filterPushdown</code></td>
  <td>true</td>
  <td>Enables Parquet filter push-down optimization when set to true.</td>
</tr>
<tr>
  <td><code>spark.sql.hive.convertMetastoreParquet</code></td>
  <td>true</td>
  <td>
    When set to false, Spark SQL will use the Hive SerDe for parquet tables instead of the built in support.
  </td>
</tr>
<tr>
  <td><code>spark.sql.parquet.mergeSchema</code></td>
  <td>false</td>
  <td>
    <p>
      When true, the Parquet data source merges schemas collected from all data files, otherwise the schema is picked from the summary file or a random data file if no summary file is available.
    </p>
  </td>
</tr>
</table>

<h2>JSON Datasets</h2>
<p>Spark SQL can automatically infer the schema of a JSON dataset and load it as a DataFrame.This conversion can be done using <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=json#pyspark.sql.DataFrameReader.json" target="_blank"><code>SparkSession.read.json</code></a> on a JSON file.</p>
<p>Note that the file that is offered as <em>a json file</em> is not a typical JSON file. Each line must contain a separate, self-contained valid JSON object. As a consequence(因而,结果),a regular multi-line JSON file will most often fail.</p>

In [29]:
peopleDF = spark.read.json('resources/people.json')
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [30]:
# Creates a temporary view using the DataFrame
peopleDF.createOrReplaceTempView("people")
# SQL statements can be run by using the sql methods provided by spark
teenagerNamesDF = spark.sql("SELECT name FROM people WHERE age BETWEEN 13 AND 19")
teenagerNamesDF.show()

+------+
|  name|
+------+
|Justin|
+------+



In [31]:
# Alternatively, a DataFrame can be created for a JSON dataset represented by
# an RDD[String] storing one JSON object per string
jsonStrings = ['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}']
otherPeopleRDD = sc.parallelize(jsonStrings)
otherPeople = spark.read.json(otherPeopleRDD)
otherPeople.show()

+---------------+----+
|        address|name|
+---------------+----+
|[Columbus,Ohio]| Yin|
+---------------+----+



In [32]:
otherPeopleRDD.collect()

['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}']

<h2>Hive Tables</h2>

<p>Spark SQL also supports reading and writing data stored in <a href="http://hive.apache.org/">Apache Hive</a>.
However, since Hive has a large number of dependencies, these dependencies are not included in the default Spark distribution. If Hive dependencies can be found on the classpath, Spark will load them automatically. Note that these Hive dependencies must also be present on all of the worker nodes, as they will need access to the Hive serialization and deserialization libraries (SerDes) in order to access data stored in Hive.</p>

<p>Configuration of Hive is done by placing your <strong>hive-site.xml</strong>, <strong>core-site.xml</strong> (for security configuration),and <strong>hdfs-site.xml</strong> (for HDFS configuration) file in <code>conf/</code>.</p>

<p>When working with Hive, one must instantiate(实例化) <code>SparkSession</code> with Hive support, including connectivity to a persistent Hive metastore, support for Hive SerDes, and Hive user-defined functions. Users who do not have an existing Hive deployment can still enable Hive support. When not configured by the <strong>hive-site.xml</strong>, the context automatically creates <code>metastore_db</code> in the current directory and creates a directory configured by <strong>spark.sql.warehouse.dir</strong>, which defaults to the directory <strong>spark-warehouse</strong> in the current directory that the spark application is started. Note that the <strong>hive.metastore.warehouse.dir</strong> property in <strong>hive-site.xml</strong> is deprecated(不赞成) since Spark 2.0.0. Instead, use <strong>spark.sql.warehouse.dir</strong> to specify the default location of database in warehouse(仓库).You may need to grant write privilege to the user who starts the spark application.</p>

In [33]:
from os.path import expanduser, join

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'file:${system:user.dir}/spark-warehouse'

spark = SparkSession \
        .builder \
        .appName("Python Spark SQL HIVE integration example") \
        .config("spark.sql.warehouse.dir", warehouse_location) \
        .enableHiveSupport()\
        .getOrCreate()
# spark is an existing SparkSession        
spark.sql("CREATE TABLE IF NOT EXISTS src(key INT, value STRING)")
spark.sql("LOAD DATA LOCAL INPATH 'resources/kv1.txt' INTO TABLE src")
# Queries are expressed in HiveQL
spark.sql("SELECT * FROM src").show()

+---+-------+
|key|  value|
+---+-------+
|238|val_238|
| 86| val_86|
|311|val_311|
| 27| val_27|
|165|val_165|
|409|val_409|
|255|val_255|
|278|val_278|
| 98| val_98|
|484|val_484|
|265|val_265|
|193|val_193|
|401|val_401|
|150|val_150|
|273|val_273|
|224|val_224|
|369|val_369|
| 66| val_66|
|128|val_128|
|213|val_213|
+---+-------+
only showing top 20 rows



In [34]:
spark.sql("SELECT COUNT(*) FROM src").show()

+--------+
|count(1)|
+--------+
|    2000|
+--------+



In [35]:
sqlDF = spark.sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")

# The items in DaraFrames are of type Row, which allows you to access each column by ordinal.
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)

Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 2, Value: val_2
Key: 2, Value: val_2
Key: 2, Value: val_2
Key: 2, Value: val_2
Key: 4, Value: val_4
Key: 4, Value: val_4
Key: 4, Value: val_4
Key: 4, Value: val_4
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 8, Value: val_8
Key: 8, Value: val_8
Key: 8, Value: val_8
Key: 8, Value: val_8
Key: 9, Value: val_9
Key: 9, Value: val_9
Key: 9, Value: val_9
Key: 9, Value: val_9


In [36]:
# You can also use DataFrames to create temporary views within a SparkSession.
Record = Row("key", "value")
recordsDF = spark.createDataFrame(map(lambda i: Record(i, 'val_' + str(i)), range(1, 101)))
recordsDF.createOrReplaceTempView("records")
# Queries can then join DataFrame data with data stored in Hive.
spark.sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

+---+-----+---+-----+
|key|value|key|value|
+---+-----+---+-----+
|  2|val_2|  2|val_2|
|  2|val_2|  2|val_2|
|  2|val_2|  2|val_2|
|  2|val_2|  2|val_2|
|  4|val_4|  4|val_4|
|  4|val_4|  4|val_4|
|  4|val_4|  4|val_4|
|  4|val_4|  4|val_4|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
|  5|val_5|  5|val_5|
+---+-----+---+-----+
only showing top 20 rows



<h3>Interacting with Different Versions of Hive Metastore</h3>

<p>One of the most important pieces of Spark SQL&#8217;s Hive support is interaction with Hive metastore,which enables Spark SQL to access metadata of Hive tables. Starting from Spark 1.4.0, a single binary build of Spark SQL can be used to query different versions of Hive metastores, using the configuration described below.<font color="red">Note that independent of the version of Hive that is being used to talk to the metastore, internally Spark SQL will compile against Hive 1.2.1 and use those classes for internal execution (serdes, UDFs, UDAFs, etc)</font>.</p>

<p>The following options can be used to configure the version of Hive that is used to retrieve metadata:</p>
<table>
  <tr><th>Property Name</th><th>Default</th><th>Meaning</th></tr>
  <tr>
    <td><code>spark.sql.hive.metastore.version</code></td>
    <td><code>1.2.1</code></td>
    <td>
    Version of the Hive metastore. Available options are <code>0.12.0</code> through <code>1.2.1</code>.
    </td>
  </tr>
  <tr>
    <td><code>spark.sql.hive.metastore.jars</code></td>
    <td><code>builtin</code></td>
    <td>
    Location of the jars that should be used to instantiate the HiveMetastoreClient. This property can be one of three options:
      <ol>
        <li><code>builtin</code></li>
        Use Hive 1.2.1, which is bundled with the Spark assembly when <code>-Phive</code> is enabled. When this option is chosen, <code>spark.sql.hive.metastore.version</code> must be either <code>1.2.1</code> or not defined.
        <li><code>maven</code></li>
        Use Hive jars of specified version downloaded from Maven repositories. This configuration is not generally recommended for production deployments.
        <li>A classpath in the standard format for the JVM. This classpath must include all of Hive and its dependencies, including the correct version of Hadoop. These jars only need to be present on the driver, but if you are running in yarn cluster mode then you must ensure they are packaged with you application.</li>
      </ol>
    </td>
  </tr>
  <tr>
    <td><code>spark.sql.hive.metastore.sharedPrefixes</code></td>
    <td><code>com.mysql.jdbc,<br />org.postgresql,<br />com.microsoft.sqlserver,<br />oracle.jdbc</code></td>
    <td>
      <p>
      A comma separated list of class prefixes that should be loaded using the classloader that is shared between Spark SQL and a specific version of Hive. An example of classes that should be shared is JDBC drivers that are needed to talk to the metastore. Other classes that need to be shared are those that interact with classes that are already shared. For example, custom appenders that are used by log4j.
      </p>
    </td>
  </tr>
  <tr>
    <td><code>spark.sql.hive.metastore.barrierPrefixes</code></td>
    <td><code>(empty)</code></td>
    <td>
      <p>
      A comma separated list of class prefixes that should explicitly be reloaded for each version of Hive that Spark SQL is communicating with. For example, Hive UDFs that are declared in a prefix that typically would be shared (i.e. <code>org.apache.spark.*</code>).
      </p>
    </td>
  </tr>
</table>

<h2>JDBC To Other Databases</h2>

<p>Spark SQL also includes a data source that can read data from other databases using JDBC(Java数据库连接Java Database Connectivity). This functionality should be preferred over using <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.JdbcRDD">JdbcRDD</a>.This is because the results are returned as a DataFrame and they can easily be processed in Spark SQL or joined with other data sources.The JDBC data source is also easier to use from Java or Python as it does not require the user to provide a ClassTag.(Note that this is different than the Spark SQL JDBC server, which allows other applications to run queries using Spark SQL).</p>

<p>To get started you will need to include the JDBC driver for you particular database on the spark classpath. For example, to connect to postgres from the Spark Shell you would run the following command:</p>

>bin/spark-shell --driver-class-path postgresql-9.4.1207.jar --jars postgresql-9.4.1207.jar

<p>在pyspark中连接mysql：(在mysql官网下载的<a href="http://dev.mysql.com/downloads/connector/j/" target="_blank">mysql-connector-java</a>解压放在/usr/local/路径下)</p>

>pyspark --driver-class-path /usr/local/mysql-connector-java-5.0.8/mysql-connector-java-5.0.8-bin.jar --jars /usr/local/mysql-connector-java-5.0.8/mysql-connector-java-5.0.8-bin.jar

<p>Tables from the remote database can be loaded as a DataFrame or Spark SQL Temporary table using the Data Sources API. The following options are supported:</p>

<table>
  <tr><th>Property Name</th><th>Meaning</th></tr>
  <tr>
    <td><code>url</code></td>
    <td>
      The <a href="http://blog.csdn.net/ring0hx/article/details/6152528" target="_blank">JDBC URL</a> to connect to.
    </td>
  </tr>
  <tr>
    <td><code>dbtable</code></td>
    <td>
      The JDBC table that should be read. Note that anything that is valid in a <code>FROM</code> clause of
      a SQL query can be used. For example, instead of a full table you could also use a
      subquery in parentheses.
    </td>
  </tr>

  <tr>
    <td><code>driver</code></td>
    <td>
      The class name of the JDBC driver to use to connect to this URL.
    </td>
  </tr>

  <tr>
    <td><code>partitionColumn, lowerBound, upperBound, numPartitions</code></td>
    <td>
      These options must all be specified if any of them is specified. They describe how to
      partition the table when reading in parallel from multiple workers.
      <code>partitionColumn</code> must be a numeric column from the table in question. Notice
      that <code>lowerBound</code> and <code>upperBound</code> are just used to decide the
      partition stride, not for filtering the rows in table. So all rows in the table will be
      partitioned and returned.
    </td>
  </tr>

  <tr>
    <td><code>fetchsize</code></td>
    <td>
      The JDBC fetch size, which determines how many rows to fetch per round trip. This can help performance on JDBC drivers which default to low fetch size (eg. Oracle with 10 rows).
    </td>
  </tr>
</table>

<font color="red">下面例子已经改成mysql，注意，在mysql中创建spark——test数据库，并在此数据库中创建表people。</font>

In [55]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option('url', 'jdbc:mysql://127.0.0.1:3306/spark_test') \
    .option('driver', 'com.mysql.jdbc.Driver') \
    .option("dbtable", "spark_test.people") \
    .option("user", "root") \
    .option("password", "2038793") \
    .load()         

<h2>Troubleshooting(故障排除)</h2>

<ul>
  <li>The JDBC driver class must be visible to the `primordial class loader`(启动类加载器) on the client session and on all executors. This is because Java&#8217;s DriverManager class does a security check that results in it ignoring all drivers not visible to the primordial class loader when one goes to open a connection. One convenient way to do this is to modify compute_classpath.sh on all worker nodes to include your driver JARs.</li>
  <li>Some databases, such as H2, convert all names to upper case. You&#8217;ll need to use upper case to refer to those names in Spark SQL.</li>
</ul>
<h1>Performance Tuning(性能调优)</h1>

<p>For some workloads it is possible to improve performance by either caching data in memory, or by turning on some experimental options(试验选项).</p>

<h2>Caching Data In Memory(缓存数据至内存)</h2>

<p>Spark SQL can cache tables using an in-memory columnar format by calling <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=cachetable#pyspark.sql.SQLContext.cacheTable" target="_blank"><code>spark.cacheTable("tableName")</code></a> or <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=cache#pyspark.sql.DataFrame.cache"><code>dataFrame.cache()</code></a>.Then Spark SQL will scan only required columns and will automatically tune compression to minimize memory usage and GC pressure. You can call <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=cachetable#pyspark.sql.SQLContext.uncacheTable"><code>spark.uncacheTable("tableName")</code></a> to remove the table from memory.</p>

<p>Configuration of in-memory caching can be done using the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=setconf#pyspark.sql.SQLContext.setConf" target="_blank"><code>setConf</code></a> method on <strong>SparkSession</strong> or by running <strong>SET key=value</strong> commands using SQL.</p>

<table>
<tr><th>Property Name</th><th>Default</th><th>Meaning</th></tr>
<tr>
  <td><code>spark.sql.inMemoryColumnarStorage.compressed</code></td>
  <td>true</td>
  <td>
    When set to true Spark SQL will automatically select a compression codec for each column based on statistics of the data.
  </td>
</tr>
<tr>
  <td><code>spark.sql.inMemoryColumnarStorage.batchSize</code></td>
  <td>10000</td>
  <td>
    Controls the size of batches for columnar caching. Larger batch sizes can improve memory utilization and compression, but risk OOMs when caching data.
  </td>
</tr>
</table>
<h2>Other Configuration Options</h2>

<p>The following options can also be used to tune the performance of query execution. It is possible that these options will be deprecated in future release as more optimizations are performed automatically.</p>

<table>
  <tr><th>Property Name</th><th>Default</th><th>Meaning</th></tr>
  <tr>
    <td><code>spark.sql.files.maxPartitionBytes</code></td>
    <td>134217728 (128 MB)</td>
    <td>
      The maximum number of bytes to pack into a single partition when reading files.
    </td>
  </tr>
  <tr>
    <td><code>spark.sql.files.openCostInBytes</code></td>
    <td>4194304 (4 MB)</td>
    <td>
      The estimated cost to open a file, measured by the number of bytes could be scanned in the same time. This is used when putting multiple files into a partition. It is better to over estimated, then the partitions with small files will be faster than partitions with bigger files (which is scheduled first).
    </td>
  </tr>
  <tr>
    <td><code>spark.sql.broadcastTimeout</code></td>
    <td>300</td>
    <td>
    <p>
      Timeout in seconds for the broadcast wait time in broadcast joins
    </p>
    </td>
  </tr>
  <tr>
    <td><code>spark.sql.autoBroadcastJoinThreshold</code></td>
    <td>10485760 (10 MB)</td>
    <td>
      Configures the maximum size in bytes for a table that will be broadcast to all worker nodes when performing a join. By setting this value to -1 broadcasting can be disabled. Note that currently statistics are only supported for Hive Metastore tables where the command
      <code>ANALYZE TABLE &lt;tableName&gt; COMPUTE STATISTICS noscan</code> has been run.
    </td>
  </tr>
  <tr>
    <td><code>spark.sql.shuffle.partitions</code></td>
    <td>200</td>
    <td>
      Configures the number of partitions to use when shuffling data for joins or aggregations.
    </td>
  </tr>
</table>
<h1>Distributed SQL Engine(分布式SQL引擎)</h1>

<p>Spark SQL can also act as a distributed query engine using its JDBC/ODBC or command-line interface.In this mode, end-users or applications can interact with Spark SQL directly to run SQL queries, without the need to write any code.</p>

<h2>Running the Thrift JDBC/ODBC server(运行Thrift JDBC/ODBC服务)</h2>

<p>The Thrift JDBC/ODBC server implemented here corresponds to the <a href="https://cwiki.apache.org/confluence/display/Hive/Setting+Up+HiveServer2"><code>HiveServer2</code></a> in Hive 1.2.1 You can test the JDBC server with the beeline script that comes with either Spark or Hive 1.2.1.</p>

<p>To start the JDBC/ODBC server, run the following in the Spark directory:</p>

<pre><strong>./sbin/start-thriftserver.sh</strong></pre>

<p>This script accepts all <strong>bin/spark-submit</strong> command line options, plus a <code>--hiveconf</code> option to specify Hive properties. You may run <strong>./sbin/start-thriftserver.sh --help</strong> for a complete list of all available options. By default, the server listens on localhost:10000. You may override this behaviour via either environment variables, i.e.:</p>
<pre><strong>export HIVE_SERVER2_THRIFT_PORT=&lt;listening-port&gt;
export HIVE_SERVER2_THRIFT_BIND_HOST=&lt;listening-host&gt;
./sbin/start-thriftserver.sh \
  --master &lt;master-uri&gt; \
  ...</strong></pre>
<p>or system properties:</p>
<pre><strong>./sbin/start-thriftserver.sh \
  --hiveconf hive.server2.thrift.port=&lt;listening-port&gt; \
  --hiveconf hive.server2.thrift.bind.host=&lt;listening-host&gt; \
  --master &lt;master-uri&gt;
  ...</strong></pre>
<p>or system properties:</p>
<pre><strong>./sbin/start-thriftserver.sh \
  --hiveconf hive.server2.thrift.port=&lt;listening-port&gt; \
  --hiveconf hive.server2.thrift.bind.host=&lt;listening-host&gt; \
  --master &lt;master-uri&gt;
  ...</strong></pre>
<p>Now you can use beeline to test the Thrift JDBC/ODBC server:</p>

<pre><strong>./bin/beeline</strong></pre>

<p>Connect to the JDBC/ODBC server in beeline with:</p>

<pre><strong>beeline&gt; !connect jdbc:hive2://localhost:10000</strong></pre>

<p>Beeline will ask you for a username and password. In non-secure mode, simply enter the username on your machine and a blank password. For secure mode, please follow the instructions given in the
<a href="https://cwiki.apache.org/confluence/display/Hive/HiveServer2+Clients">beeline documentation</a>.</p>

<p>Configuration of Hive is done by placing your <strong>hive-site.xml</strong>, <strong>core-site.xml</strong> and <strong>hdfs-site.xml</strong> files in <strong>conf/</strong>.</p>

<p>You may also use the beeline script that comes with Hive.</p>

<p>Thrift JDBC server also supports sending thrift RPC messages over HTTP transport.Use the following setting to enable HTTP mode as system property or in <strong>hive-site.xml</strong> file in <strong>conf/</strong>:</p>

<p>You may also use the beeline script that comes with Hive.</p>

<p>Thrift JDBC server also supports sending thrift RPC messages over HTTP transport.Use the following setting to enable HTTP mode as system property or in <strong>hive-site.xml</strong> file in <strong>conf/</strong>:</p>

<pre><strong>hive.server2.transport.mode - Set this to value: http
hive.server2.thrift.http.port - HTTP port number fo listen on; default is 10001
hive.server2.http.endpoint - HTTP endpoint; default is cliservice
</strong></pre>

<p>To test, use beeline to connect to the JDBC/ODBC server in http mode with:</p>

<pre><strong>beeline&gt; !connect jdbc:hive2://&lt;host&gt;:&lt;port&gt;/&lt;database&gt;?hive.server2.transport.mode=http;hive.server2.thrift.http.path=&lt;http_endpoint&gt;</strong></pre>

<h2>Running the Spark SQL CLI</h2>

<p>The Spark SQL CLI is a convenient tool to run the Hive metastore service in local mode and execute queries input from the command line. Note that the Spark SQL CLI cannot talk to the Thrift JDBC server.</p>

<p>To start the Spark SQL CLI, run the following in the Spark directory:</p>

<pre><strong>./bin/spark-sql</strong></pre>

<p>Configuration of Hive is done by placing your <strong>hive-site.xml</strong>, <strong>core-site.xml</strong> and <strong>hdfs-site.xml</strong> files in <strong>conf/</strong>.You may run <strong>./bin/spark-sql --help</strong> for a complete list of all available options.</p>

<h1>Migration Guide</h1>

<h2>Upgrading From Spark SQL 1.6 to 2.0</h2>

<ul>
  <li>
    <p><code>SparkSession</code> is now the new entry point of Spark that replaces the old <code>SQLContext</code> and
<code>HiveContext</code>. Note that the old SQLContext and HiveContext are kept for backward compatibility. A new <code>catalog</code> interface is accessible from <code>SparkSession</code> - existing API on databases and tables access such as <code>listTables</code>, <code>createExternalTable</code>, <code>dropTempView</code>, <code>cacheTable</code> are moved here.</p>
  </li>
  <li>
    <p>Dataset API and DataFrame API are unified. In Scala, <code>DataFrame</code> becomes a type alias for
<code>Dataset[Row]</code>, while Java API users must replace <code>DataFrame</code> with <code>Dataset&lt;Row&gt;</code>. Both the typed transformations (e.g. <code>map</code>, <code>filter</code>, and <code>groupByKey</code>) and untyped transformations (e.g.<code>select</code> and <code>groupBy</code>) are available on the Dataset class. Since compile-time type-safety in Python and R is not a language feature, the concept of Dataset does not apply to these languages’APIs. Instead, <code>DataFrame</code> remains the primary programing abstraction, which is analogous to the
single-node data frame notion in these languages.</p>
  </li>
  <li>Dataset and DataFrame API <code>unionAll</code> has been deprecated and replaced by <code>union</code></li>
  <li>Dataset and DataFrame API <code>explode</code> has been deprecated, alternatively, use <code>functions.explode()</code> with <code>select</code> or <code>flatMap</code></li>
  <li>Dataset and DataFrame API <code>registerTempTable</code> has been deprecated and replaced by <code>createOrReplaceTempView</code></li>
</ul>

<h2>Upgrading From Spark SQL 1.5 to 1.6</h2>

<ul>
  <li>From Spark 1.6, by default the Thrift server runs in multi-session mode. Which means each JDBC/ODBC connection owns a copy of their own SQL configuration and temporary function registry. Cached tables are still shared though. If you prefer to run the Thrift server in the old single-session mode, please set option <code>spark.sql.hive.thriftServer.singleSession</code> to <code>true</code>. You may either add this option to <code>spark-defaults.conf</code>, or pass it to <code>start-thriftserver.sh</code> via <code>--conf</code>:</li>
</ul>
<pre><strong>./sbin/start-thriftserver.sh \
     --conf spark.sql.hive.thriftServer.singleSession=true \
     ...</strong></pre>

<ul>
  <li>
    <p>Since 1.6.1, withColumn method in sparkR supports adding a new column to or replacing existing columns
of the same name of a DataFrame.</p>
  </li>
  <li>
    <p>From Spark 1.6, LongType casts to TimestampType expect seconds instead of microseconds. This
change was made to match the behavior of Hive 1.2 for more consistent type casting to TimestampType
from numeric types. See <a href="https://issues.apache.org/jira/browse/SPARK-11724">SPARK-11724</a> for
details.</p>
  </li>
</ul>

<h2>Upgrading From Spark SQL 1.4 to 1.5</h2>

<ul>
  <li>Optimized execution using manually managed memory (Tungsten) is now enabled by default, along with
code generation for expression evaluation. These features can both be disabled by setting <code>spark.sql.tungsten.enabled</code> to <code>false</code>.</li>
  <li>Parquet schema merging is no longer enabled by default. It can be re-enabled by setting
<code>spark.sql.parquet.mergeSchema</code> to <code>true</code>.</li>
  <li>Resolution of strings to columns in python now supports using dots (<code>.</code>) to qualify the column or
access nested values. For example <code>df['table.column.nestedField']</code>. However, this means that if
your column name contains any dots you must now escape them using backticks (e.g., <code>table.`column.with.dots`.nested</code>).</li>
  <li>In-memory columnar storage partition pruning is on by default. It can be disabled by setting
<code>spark.sql.inMemoryColumnarStorage.partitionPruning</code> to <code>false</code>.</li>
  <li>Unlimited precision decimal columns are no longer supported, instead Spark SQL enforces a maximum
precision of 38. When inferring schema from <code>BigDecimal</code> objects, a precision of (38, 18) is now
used. When no precision is specified in DDL then the default remains <code>Decimal(10, 0)</code>.</li>
  <li>Timestamps are now stored at a precision of 1us, rather than 1ns</li>
  <li>In the <code>sql</code> dialect, floating point numbers are now parsed as decimal. HiveQL parsing remains
unchanged.</li>
  <li>The canonical name of SQL/DataFrame functions are now lower case (e.g. sum vs SUM).</li>
  <li>JSON data source will not automatically load new files that are created by other applications
(i.e. files that are not inserted to the dataset through Spark SQL).
For a JSON persistent table (i.e. the metadata of the table is stored in Hive Metastore),
users can use <code>REFRESH TABLE</code> SQL command or <code>HiveContext</code>&#8217;s <code>refreshTable</code> method
to include those new files to the table. For a DataFrame representing a JSON dataset, users need to recreate
the DataFrame and the new DataFrame will include new files.</li>
  <li>DataFrame.withColumn method in pySpark supports adding a new column or replacing existing columns of the same name.</li>
</ul>

<h2>Upgrading from Spark SQL 1.3 to 1.4</h2>

<h4>DataFrame data reader/writer interface</h4>

<p>Based on user feedback, we created a new, more fluid API for reading data in (<code>SQLContext.read</code>)
and writing data out (<code>DataFrame.write</code>),and deprecated the old APIs (e.g. <code>SQLContext.parquetFile</code>, <code>SQLContext.jsonFile</code>).</p>

<p>See the API docs for <code>SQLContext.read</code> (
  <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.SQLContext@read:DataFrameReader">Scala</a>,
  <a href="http://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/SQLContext.html#read()">Java</a>,
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.read">Python</a>
) and <code>DataFrame.write</code> (
  <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.SQLContext@read:DataFrameReader">Scala</a>,
  <a href="http://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/SQLContext.html#read()">Java</a>,
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.read">Python</a>
) more information.</p>

<h4>DataFrame.groupBy retains grouping columns</h4>

<p>Based on user feedback, we changed the default behavior of <code>DataFrame.groupBy().agg()</code> to retain the
grouping columns in the resulting <code>DataFrame</code>. To keep the behavior in 1.3, set <code>spark.sql.retainGroupColumns</code> to <code>false</code>.</p>

In [57]:
import pyspark.sql.functions as func

# In 1.3.x, in order for the grouping column "department" to show up,
# it must be included explicitly as part of the agg function call.
#df.groupBy("department").agg(df["department"], func.max("age"), func.sum("expense"))

# In 1.4+, grouping column "department" is included automatically.
#df.groupBy("department").agg(func.max("age"), func.sum("expense"))

# Revert to 1.3.x behavior (not retaining grouping column) by:
#sqlContext.setConf("spark.sql.retainGroupColumns", "false")

<h4>Behavior change on DataFrame.withColumn</h4>

<p>Prior to 1.4, DataFrame.withColumn() supports adding a column only. The column will always be added as a new column with its specified name in the result DataFrame even if there may be any existing columns of the same name. Since 1.4, DataFrame.withColumn() supports adding a column of a different name from names of all existing columns or replacing existing columns of the same name.</p>

<p>Note that this change is only for Scala API, not for PySpark and SparkR.</p>

<h2>Upgrading from Spark SQL 1.0-1.2 to 1.3</h2>

<p>In Spark 1.3 we removed the &#8220;Alpha&#8221; label from Spark SQL and as part of this did a cleanup of the
available APIs. From Spark 1.3 onwards, Spark SQL will provide binary compatibility with other releases in the 1.X series. This compatibility guarantee excludes APIs that are explicitly marked as unstable (i.e., DeveloperAPI or Experimental).</p>

<h4>Rename of SchemaRDD to DataFrame</h4>

<p>The largest change that users will notice when upgrading to Spark SQL 1.3 is that <code>SchemaRDD</code> has
been renamed to <code>DataFrame</code>. This is primarily because DataFrames no longer inherit from RDD directly, but instead provide most of the functionality that RDDs provide though their own implementation. DataFrames can still be converted to RDDs by calling the <code>.rdd</code> method.</p>

<p>In Scala there is a type alias from <code>SchemaRDD</code> to <code>DataFrame</code> to provide source compatibility for some use cases. It is still recommended that users update their code to use <code>DataFrame</code> instead. Java and Python users will need to update their code.</p>

<h4>Unification of the Java and Scala APIs</h4>
<p>Prior to Spark 1.3 there were separate Java compatible classes (<code>JavaSQLContext</code> and <code>JavaSchemaRDD</code>) hat mirrored the Scala API. In Spark 1.3 the Java API and Scala API have been unified. Users
of either language should use <code>SQLContext</code> and <code>DataFrame</code>. In general theses classes try to
use types that are usable from both languages (i.e. <code>Array</code> instead of language specific collections).
In some cases where no common type exists (e.g., for passing in closures or Maps) function overloading is used instead.</p>

<p>Additionally the Java specific types API has been removed. Users of both Scala and Java should use the classes present in <code>org.apache.spark.sql.types</code> to describe schema programmatically.</p>

<h4>Isolation of Implicit Conversions and Removal of dsl Package (Scala-only)</h4>

<p>Many of the code examples prior to Spark 1.3 started with <code>import sqlContext._</code>, which brought
all of the functions from sqlContext into scope. In Spark 1.3 we have isolated the implicit conversions for converting <code>RDD</code>s into <code>DataFrame</code>s into an object inside of the <code>SQLContext</code>.Users should now write <code>import sqlContext.implicits._</code>.</p>

<p>Additionally, the implicit conversions now only augment RDDs that are composed of <code>Product</code>s (i.e.,
case classes or tuples) with a method <code>toDF</code>, instead of applying automatically.</p>

<p>When using function inside of the DSL (now replaced with the <code>DataFrame</code> API) users used to import
<code>org.apache.spark.sql.catalyst.dsl</code>. Instead the public dataframe functions API should be used:
<code>import org.apache.spark.sql.functions._</code>.</p>

<h4>Removal of the type aliases in org.apache.spark.sql for DataType (Scala-only)</h4>

<p>Spark 1.3 removes the type aliases that were present in the base sql package for <code>DataType</code>. Users
should instead import the classes in <code>org.apache.spark.sql.types</code></p>

<h4>UDF Registration Moved to <code>sqlContext.udf</code> (Java &amp; Scala)</h4>

<p>Functions that are used to register UDFs, either for use in the DataFrame DSL or SQL, have been moved into the udf object in <code>SQLContext</code>.</p>
<p>Python UDF registration is unchanged.</p>

<h4>Python DataTypes No Longer Singletons</h4>

<p>When using DataTypes in Python you will need to construct them (i.e. <code>StringType()</code>) instead of referencing a singleton.</p>

<h2>Compatibility with Apache Hive(兼容Hive)</h2>

<p>Spark SQL is designed to be compatible with the Hive Metastore, SerDes and UDFs.Currently Hive SerDes and UDFs are based on Hive 1.2.1,and Spark SQL can be connected to different versions of Hive Metastore (from 0.12.0 to 1.2.1. Also see [Interacting with Different Versions of Hive Metastore] (#interacting-with-different-versions-of-hive-metastore)).</p>

<h4>Deploying in Existing Hive Warehouses(在Hive warehouse中部署Spark SQL)</h4>

<p>The Spark SQL Thrift JDBC server is designed to be &#8220;out of the box&#8221; compatible with existing Hive
installations. You do not need to modify your existing Hive Metastore or change the data placement or partitioning of your tables.</p>

<h3>Supported Hive Features(支持的Hive特性)</h3>

<p>Spark SQL supports the vast majority of Hive features, such as:</p>

<ul>
  <li>Hive query statements, including:
    <ul>
      <li><code>SELECT</code></li>
      <li><code>GROUP BY</code></li>
      <li><code>ORDER BY</code></li>
      <li><code>CLUSTER BY</code></li>
      <li><code>SORT BY</code></li>
    </ul>
  </li>
  <li>All Hive operators, including:
    <ul>
      <li>Relational operators (<code>=</code>, <code>⇔</code>, <code>==</code>, <code>&lt;&gt;</code>, <code>&lt;</code>, <code>&gt;</code>, <code>&gt;=</code>, <code>&lt;=</code>, etc)</li>
      <li>Arithmetic operators (<code>+</code>, <code>-</code>, <code>*</code>, <code>/</code>, <code>%</code>, etc)</li>
      <li>Logical operators (<code>AND</code>, <code>&amp;&amp;</code>, <code>OR</code>, <code>||</code>, etc)</li>
      <li>Complex type constructors</li>
      <li>Mathematical functions (<code>sign</code>, <code>ln</code>, <code>cos</code>, etc)</li>
      <li>String functions (<code>instr</code>, <code>length</code>, <code>printf</code>, etc)</li>
    </ul>
  </li>
  <li>User defined functions (UDF)</li>
  <li>User defined aggregation functions (UDAF)</li>
  <li>User defined serialization formats (SerDes)</li>
  <li>Window functions</li>
  <li>Joins
    <ul>
      <li><code>JOIN</code></li>
      <li><code>{LEFT|RIGHT|FULL} OUTER JOIN</code></li>
      <li><code>LEFT SEMI JOIN</code></li>
      <li><code>CROSS JOIN</code></li>
    </ul>
  </li>
  <li>Unions</li>
  <li>Sub-queries
    <ul>
      <li><code>SELECT col FROM ( SELECT a + b AS col from t1) t2</code></li>
    </ul>
  </li>
  <li>Sampling</li>
  <li>Explain</li>
  <li>Partitioned tables including dynamic partition insertion</li>
  <li>View</li>
  <li>All Hive DDL Functions, including:
    <ul>
      <li><code>CREATE TABLE</code></li>
      <li><code>CREATE TABLE AS SELECT</code></li>
      <li><code>ALTER TABLE</code></li>
    </ul>
  </li>
  <li>Most Hive Data types, including:
    <ul>
      <li><code>TINYINT</code></li>
      <li><code>SMALLINT</code></li>
      <li><code>INT</code></li>
      <li><code>BIGINT</code></li>
      <li><code>BOOLEAN</code></li>
      <li><code>FLOAT</code></li>
      <li><code>DOUBLE</code></li>
      <li><code>STRING</code></li>
      <li><code>BINARY</code></li>
      <li><code>TIMESTAMP</code></li>
      <li><code>DATE</code></li>
      <li><code>ARRAY&lt;&gt;</code></li>
      <li><code>MAP&lt;&gt;</code></li>
      <li><code>STRUCT&lt;&gt;</code></li>
    </ul>
  </li>
</ul>

<h3>Unsupported Hive Functionality(不支持的Hive功能)</h3>

<p>Below is a list of Hive features that we don&#8217;t support yet. Most of these features are rarely used in Hive deployments.</p>

<p><strong>Major Hive Features</strong></p>

<ul>
  <li>Tables with buckets: bucket is the hash partitioning within a Hive table partition. Spark SQL doesn&#8217;t support buckets yet.</li>
</ul>

<p><strong>Esoteric Hive Features</strong></p>

<ul>
  <li><code>UNION</code> type</li>
  <li>Unique join</li>
  <li>Column statistics collecting: Spark SQL does not piggyback scans to collect column statistics at
the moment and only supports populating the sizeInBytes field of the hive metastore.</li>
</ul>

<p><strong>Hive Input/Output Formats</strong></p>

<ul>
  <li>File format for CLI: For results showing back to the CLI, Spark SQL only supports TextOutputFormat.</li>
  <li>Hadoop archive</li>
</ul>

<p><strong>Hive Optimizations</strong></p>

<p>A handful of Hive optimizations are not yet included in Spark. Some of these (such as indexes) are less important due to Spark SQL&#8217;s in-memory computational model. Others are slotted for future releases of Spark SQL.</p>

<ul>
  <li>Block level bitmap indexes and virtual columns (used to build indexes)</li>
  <li>Automatically determine the number of reducers for joins and groupbys: Currently in Spark SQL, you need to control the degree of parallelism post-shuffle using &#8220;<code>SET spark.sql.shuffle.partitions=[num_tasks];</code>&#8221;.</li>
  <li>Meta-data only query: For queries that can be answered by using only meta data, Spark SQL still launches tasks to compute the result.</li>
  <li>Skew data flag: Spark SQL does not follow the skew data flags in Hive.</li>
  <li><code>STREAMTABLE</code> hint in join: Spark SQL does not follow the <code>STREAMTABLE</code> hint.</li>
  <li>Merge multiple small files for query results: if the result output contains multiple small files,Hive can optionally merge the small files into fewer large files to avoid overflowing the HDFS metadata. Spark SQL does not support that.</li>
</ul>

<h1>Reference</h1>

<h2>Data Types</h2>

<p>Spark SQL and DataFrames support the following data types:</p>

<ul>
  <li>Numeric types
    <ul>
      <li><code>ByteType</code>: Represents 1-byte signed integer numbers.The range of numbers is from <code>-128</code> to <code>127</code>.</li>
      <li><code>ShortType</code>: Represents 2-byte signed integer numbers.The range of numbers is from <code>-32768</code> to <code>32767</code>.</li>
      <li><code>IntegerType</code>: Represents 4-byte signed integer numbers.The range of numbers is from <code>-2147483648</code> to <code>2147483647</code>.</li>
      <li><code>LongType</code>: Represents 8-byte signed integer numbers.The range of numbers is from <code>-9223372036854775808</code> to <code>9223372036854775807</code>.</li>
      <li><code>FloatType</code>: Represents 4-byte single-precision floating point numbers.</li>
      <li><code>DoubleType</code>: Represents 8-byte double-precision floating point numbers.</li>
      <li><code>DecimalType</code>: Represents arbitrary-precision signed decimal numbers. Backed internally by <code>java.math.BigDecimal</code>. A <code>BigDecimal</code> consists of an arbitrary precision integer unscaled value and a 32-bit integer scale.</li>
    </ul>
  </li>
  <li>String type
    <ul>
      <li><code>StringType</code>: Represents character string values.</li>
    </ul>
  </li>
  <li>Binary type
    <ul>
      <li><code>BinaryType</code>: Represents byte sequence values.</li>
    </ul>
  </li>
  <li>Boolean type
    <ul>
      <li><code>BooleanType</code>: Represents boolean values.</li>
    </ul>
  </li>
  <li>Datetime type
    <ul>
      <li><code>TimestampType</code>: Represents values comprising values of fields year, month, day, hour, minute, and second.</li>
      <li><code>DateType</code>: Represents values comprising values of fields year, month, day.</li>
    </ul>
  </li>
  <li>Complex types
    <ul>
      <li><code>ArrayType(elementType, containsNull)</code>: Represents values comprising a sequence of elements with the type of <code>elementType</code>. <code>containsNull</code> is used to indicate if elements in a <code>ArrayType</code> value can have <code>null</code> values.</li>
      <li><code>MapType(keyType, valueType, valueContainsNull)</code>: Represents values comprising a set of key-value pairs. The data type of keys are described by <code>keyType</code> and the data type of values are described by <code>valueType</code>.For a <code>MapType</code> value, keys are not allowed to have <code>null</code> values. <code>valueContainsNull</code>is used to indicate if values of a <code>MapType</code> value can have <code>null</code> values.</li>
      <li><code>StructType(fields)</code>: Represents values with the structure described by a sequence of <code>StructField</code>s (<code>fields</code>).
        <ul>
          <li><code>StructField(name, dataType, nullable)</code>: Represents a field in a <code>StructType</code>.
The name of a field is indicated by <code>name</code>. The data type of a field is indicated by <code>dataType</code>. <code>nullable</code> is used to indicate if values of this fields can have <code>null</code> values.</li>
        </ul>
      </li>
    </ul>
  </li>
</ul>
<p>All data types of Spark SQL are located in the package of <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=sql.types#module-pyspark.sql.types"><strong>pyspark.sql.types</strong></a>.You can access them by doing:</p>

In [58]:
from pyspark.sql.types import *

<table>
<tr>
  <th style="width:20%">Data type</th>
  <th style="width:40%">Value type in Python</th>
  <th>API to access or create a data type</th></tr>
<tr>
  <td> <b>ByteType</b> </td>
  <td>
  int or long <br />
  <b>Note:</b> Numbers will be converted to 1-byte signed integer numbers at runtime.Please make sure that numbers are within the range of -128 to 127.
  </td>
  <td>
  ByteType()
  </td>
</tr>
<tr>
  <td> <b>ShortType</b> </td>
  <td>
  int or long <br />
  <b>Note:</b> Numbers will be converted to 2-byte signed integer numbers at runtime.Please make sure that numbers are within the range of -32768 to 32767.
  </td>
  <td>
  ShortType()
  </td>
</tr>
<tr>
  <td> <b>IntegerType</b> </td>
  <td> int or long </td>
  <td>
  IntegerType()
  </td>
</tr>
<tr>
  <td> <b>LongType</b> </td>
  <td>
  long <br />
  <b>Note:</b> Numbers will be converted to 8-byte signed integer numbers at runtime.Please make sure that numbers are within the range of -9223372036854775808 to 9223372036854775807.Otherwise, please convert data to decimal.Decimal and use DecimalType.
  </td>
  <td>
  LongType()
  </td>
</tr>
<tr>
  <td> <b>FloatType</b> </td>
  <td>
  float <br />
  <b>Note:</b> Numbers will be converted to 4-byte single-precision floating
  point numbers at runtime.
  </td>
  <td>
  FloatType()
  </td>
</tr>
<tr>
  <td> <b>DoubleType</b> </td>
  <td> float </td>
  <td>
  DoubleType()
  </td>
</tr>
<tr>
  <td> <b>DecimalType</b> </td>
  <td> decimal.Decimal </td>
  <td>
  DecimalType()
  </td>
</tr>
<tr>
  <td> <b>StringType</b> </td>
  <td> string </td>
  <td>
  StringType()
  </td>
</tr>
<tr>
  <td> <b>BinaryType</b> </td>
  <td> bytearray </td>
  <td>
  BinaryType()
  </td>
</tr>
<tr>
  <td> <b>BooleanType</b> </td>
  <td> bool </td>
  <td>
  BooleanType()
  </td>
</tr>
<tr>
  <td> <b>TimestampType</b> </td>
  <td> datetime.datetime </td>
  <td>
  TimestampType()
  </td>
</tr>
<tr>
  <td> <b>DateType</b> </td>
  <td> datetime.date </td>
  <td>
  DateType()
  </td>
</tr>
<tr>
  <td> <b>ArrayType</b> </td>
  <td> list, tuple, or array </td>
  <td>
  ArrayType(<i>elementType</i>, [<i>containsNull</i>])<br />
  <b>Note:</b> The default value of <i>containsNull</i> is <i>True</i>.
  </td>
</tr>
<tr>
  <td> <b>MapType</b> </td>
  <td> dict </td>
  <td>
  MapType(<i>keyType</i>, <i>valueType</i>, [<i>valueContainsNull</i>])<br />
  <b>Note:</b> The default value of <i>valueContainsNull</i> is <i>True</i>.
  </td>
</tr>
<tr>
  <td> <b>StructType</b> </td>
  <td> list or tuple </td>
  <td>
  StructType(<i>fields</i>)<br />
  <b>Note:</b> <i>fields</i> is a Seq of StructFields. Also, two fields with the same name are not allowed.
  </td>
</tr>
<tr>
  <td> <b>StructField</b> </td>
  <td> The value type in Python of the data type of this field(For example, Int for a StructField with the data type IntegerType) </td>
  <td>
  StructField(<i>name</i>, <i>dataType</i>, <i>nullable</i>)
  </td>
</tr>
</table>

<h2>NaN Semantics(NaN语义)</h2>
<p>There is specially handling for not-a-number (NaN) when dealing with <code>float</code> or <code>double</code> types that does not exactly match standard floating point semantics.Specifically(当处理float或double类型时，如果类型不符合标准的浮点语义，则使用专门的处理方式NaN。需要注意的是：):</p>
<ul>
  <li>NaN = NaN returns true(NaN = NaN 返回 true).</li>
  <li>In aggregations all NaN values are grouped together(可以对NaN值进行聚合操作).</li>
  <li>NaN is treated as a normal value in join keys(在join操作中，key为NaN时，NaN值与普通的数值处理逻辑相同).</li>
  <li>NaN values go last when in ascending order, larger than any other numeric value(NaN值大于所有的数值型数据，在升序排序中排在最后).</li>
</ul>